In [2]:
from bs4 import BeautifulSoup
import requests
import os 
import os.path

import time 
import json
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException,InvalidSessionIdException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC   
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service



In [6]:
header=['username','comment','label','date']
api="https://api-graphql.lefigaro.fr/"
id_comments="widget-comments_commentsQuery2_31d9f1fd61a3568936b76800aef3aade1b9002eee01930e2b9c499ceca28192e"

In [7]:
def start():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

In [98]:

driver=start()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/home/elkharroubi/.wdm/drivers/chromedriver/linux64/100.0.4896.60/chromedriver] found in cache


In [10]:
def get_data_content_id(driver):
    div_data_content=driver.find_element(by=By.CSS_SELECTOR, value='div.fig-comments')
    data_content_id=div_data_content.get_attribute('data-content-id')
    return data_content_id

In [11]:
def get_comments(data):
    i=1
    all_comments=[]
    comments=data['data']
    comments_data=comments['comments']
    if len(comments_data)==0:
        return all_comments
    for comment in comments_data:    
        text=comment['text']
        author=comment['author']
        date=comment['createdAt']
        if author:
            all_comments.append([author['username'],text,'',date])
        else:
            all_comments.append(['uknown',text,'',date])    
    
    return all_comments    

In [12]:

def send_request(data_content_id,page=1,id=id_comments):
    path="/graphql?id="+id_comments+"&variables={%22id%22:%22"+data_content_id+"%22,%22page%22:"+str(page)+"}"
    data_url=api+path
    response = requests.request("GET", data_url)
    return response

In [13]:

# This function we will use it to loop all the comments. page per page
def process_comments(data_content_id):
    comments_data=[]
    i=1
    while i!=0:
        response=send_request(page=i,data_content_id=data_content_id)
        if response.ok:
            raw_data=response.text

            data=json.loads(raw_data)
            # this instruction return list of comments
            comments=get_comments(data)

            if len(comments)==0:
                i=0
                continue
            else:
                for comment in comments:
                    comments_data.append(comment)
                i=i+1
                continue
        else:
            print('erreur in response')    
            i=0
            continue
    return comments_data              




In [36]:
def process_link(link,driver):
    # first we should check if there is comments on the article
    comments=[]
    driver.get(link)
    #time.sleep(2)
    com=driver.find_element(by=By.CSS_SELECTOR,value='button p.fig-comments-tabbar__small').text.strip()
    print('Number of comment ',com)
    number_comment=int(com)

    if number_comment==0 or number_comment==1:
        return comments
    # second we should get the id of every page
    # this id we give us the acces to comments
    data_content_id=get_data_content_id(driver)
    comments=process_comments(data_content_id)
    return comments


In [15]:
#links_ukraine_articles=pd.read_csv('./data/links_ukraine_articles.csv')

In [94]:
links_russie_articles=pd.read_csv('./data/links_russie_articles.csv')

In [97]:
links_russie_articles.values[2][0]

'http://www.lefigaro.fr/international/guerre-en-ukraine-les-scenarios-d-attaques-chimiques-inquietent-l-occident-20220321'

In [99]:

data_comments=[]
links_exception=[]
i=0
for link in links_russie_articles.values:
    try:
        i=i+1
        print('link number : ',i)
        comments=process_link(link[0],driver)
        if len(comments)==0:
            continue
        for comment in comments:
            data_comments.append(comment)
    except WebDriverException:
        links_exception.append(link[0])
    except InvalidSessionIdException:
        links_exception.append(link[0])
        continue

pd.DataFrame(data_comments,columns=header).to_csv('./data/russie_data.csv',index=False)

link number :  1
Number of comment  1
link number :  2
Number of comment  0
link number :  3
Number of comment  36
link number :  4
Number of comment  140
link number :  5
Number of comment  1
link number :  6
Number of comment  14
link number :  7
Number of comment  44
link number :  8
Number of comment  1
link number :  9
Number of comment  12
link number :  10
Number of comment  236
link number :  11
Number of comment  3
link number :  12
Number of comment  39
link number :  13
Number of comment  1
link number :  14
Number of comment  46
link number :  15
Number of comment  0
link number :  16
Number of comment  2
link number :  17
Number of comment  19
link number :  18
Number of comment  23
link number :  19
Number of comment  33
link number :  20
Number of comment  3
link number :  21
Number of comment  5
link number :  22
Number of comment  10
link number :  23
Number of comment  118
link number :  24
Number of comment  5
link number :  25
Number of comment  36
link number :  26

29359

In [ ]:
pd.DataFrame(data_comments,columns=header).to_csv('./data/russie_data.csv',index=False)

In [14]:
text=pd.read_csv('./data/russie_data.csv').drop_duplicates()

In [20]:
text

,username,comment,label,date
0,Salvagny 25,Les Russes n’ont pas d’armes chimiques. Point ...,NaN,2022-03-25T00:45:35.000Z
1,Nenjetplus,Il faut que Biden et les États-Unis arrêtent d...,NaN,2022-03-23T15:40:50.000Z
2,Geneviève Meunier-dumont,il faut ressuciter Powel&nbsp;!,NaN,2022-03-23T12:58:00.000Z
3,Maria Gentile,Propagande américaine exactement comme pour l'...,NaN,2022-03-23T11:49:47.000Z
4,Pierre-Louis Etienne,Et soudain on ne parle plus des laboratoires u...,NaN,2022-03-23T11:28:07.000Z
...,...,...,...,...
29354,uknown,il produise du riz aussi (les usa) subventionn...,NaN,2010-04-19T08:44:19.000Z
29355,uknown,"les epoques ont changé, le blé n'est plus auss...",NaN,2010-04-17T14:52:54.000Z
29356,uknown,Encore un crime du TMP (Très Méchant Poutine)&...,NaN,2010-04-17T09:19:21.000Z
29357,Gérald_Decastille,Brillante analyse de Mme Carrère d'Encausse à ...,NaN,2010-03-14T17:57:25.000Z


In [25]:
data=text.values

In [56]:
filtered_data=[]
for comment in data:
    if len(comment[1].split())<50:
        filtered_data.append(comment)


In [93]:
pd.DataFrame(data_comments,columns=header).to_csv('./data/ukraine_data_with_length_100.csv',index=False)